In [1]:
! pip install transformers
! pip install datasets
! pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.1 MB/s eta 0:00:00


In [1]:
import pandas as pd
import numpy as np
import datasets
import torch
#from datasets import DatasetDict, load_dataset
from transformers import TFAutoModel, AutoTokenizer
from transformers import TrainingArguments, Trainer
from transformers import BertForPreTraining
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

In [2]:
train_df = pd.read_json("/content/train_spider.json")
dev_df = pd.read_json("/content/dev.json")
print(train_df.head())
print(dev_df.head())

                   db_id                                              query  \
0  department_management         SELECT count(*) FROM head WHERE age  >  56   
1  department_management  SELECT name ,  born_state ,  age FROM head ORD...   
2  department_management  SELECT creation ,  name ,  budget_in_billions ...   
3  department_management  SELECT max(budget_in_billions) ,  min(budget_i...   
4  department_management  SELECT avg(num_employees) FROM department WHER...   

                                          query_toks  \
0  [SELECT, count, (, *, ), FROM, head, WHERE, ag...   
1  [SELECT, name, ,, born_state, ,, age, FROM, he...   
2  [SELECT, creation, ,, name, ,, budget_in_billi...   
3  [SELECT, max, (, budget_in_billions, ), ,, min...   
4  [SELECT, avg, (, num_employees, ), FROM, depar...   

                                 query_toks_no_value  \
0  [select, count, (, *, ), from, head, where, ag...   
1  [select, name, ,, born_state, ,, age, from, he...   
2  [select, creation

In [3]:
train_df = train_df.drop(['db_id', 'query_toks', 'query_toks_no_value', 'question_toks', 'sql'], axis=1)
train_df[:10]

,query,question
0,SELECT count(*) FROM head WHERE age > 56,How many heads of the departments are older th...
1,"SELECT name , born_state , age FROM head ORD...","List the name, born state and age of the heads..."
2,"SELECT creation , name , budget_in_billions ...","List the creation year, name and budget of eac..."
3,"SELECT max(budget_in_billions) , min(budget_i...",What are the maximum and minimum budget of the...
4,SELECT avg(num_employees) FROM department WHER...,What is the average number of employees of the...
5,SELECT name FROM head WHERE born_state != 'Cal...,What are the names of the heads who are born o...
6,SELECT DISTINCT T1.creation FROM department AS...,What are the distinct creation years of the de...
7,SELECT born_state FROM head GROUP BY born_stat...,What are the names of the states where at leas...
8,SELECT creation FROM department GROUP BY creat...,In which year were most departments established?
9,"SELECT T1.name , T1.num_employees FROM depart...",Show the name and number of employees for the ...


In [4]:
dev_df = dev_df.drop(['db_id', 'query_toks', 'query_toks_no_value', 'question_toks', 'sql'], axis=1)
dev_df[:10]

,query,question
0,SELECT count(*) FROM singer,How many singers do we have?
1,SELECT count(*) FROM singer,What is the total number of singers?
2,"SELECT name , country , age FROM singer ORDE...","Show name, country, age for all singers ordere..."
3,"SELECT name , country , age FROM singer ORDE...","What are the names, countries, and ages for ev..."
4,"SELECT avg(age) , min(age) , max(age) FROM s...","What is the average, minimum, and maximum age ..."
5,"SELECT avg(age) , min(age) , max(age) FROM s...","What is the average, minimum, and maximum age ..."
6,"SELECT song_name , song_release_year FROM sin...",Show the name and the release year of the song...
7,"SELECT song_name , song_release_year FROM sin...",What are the names and release years for all t...
8,SELECT DISTINCT country FROM singer WHERE age ...,What are all distinct countries where singers ...
9,SELECT DISTINCT country FROM singer WHERE age ...,What are the different countries with singers...


In [ ]:
# def tokenize(batch):
#   return tokenizer(batch, padding=True, truncation=True, return_tensors='tf')

train_df = pd.DataFrame({
    'question':train_df['question'].to_list(),
    'query':train_df['query'].to_list()
})

dev_df = pd.DataFrame({
    'question':dev_df['question'].to_list(),
    'query':dev_df['query'].to_list()
})

train_dataset = Dataset.from_dict(train_df)
test_dataset = Dataset.from_dict(dev_df)
data = datasets.DatasetDict({"train":train_dataset,"test":test_dataset})

In [5]:
model = BertForPreTraining.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [6]:
def tokenize(batch):
  return tokenizer(batch.to_list(), padding=True, truncation=True, return_tensors='tf')

In [7]:
X_train = train_df['question'].to_list()
y_train = train_df['query'].to_list()

X_test = dev_df['question'].to_list()
y_test = dev_df['query'].to_list()

In [8]:
encoded_X_train = tokenizer(X_train, padding=True, truncation=True, return_tensors='tf')
encoded_y_train = tokenizer(y_train, padding=True, truncation=True, return_tensors='tf')
encoded_X_test = tokenizer(X_test, padding=True, truncation=True, return_tensors='tf')
encoded_y_test = tokenizer(y_test, padding=True, truncation=True, return_tensors='tf')

In [13]:
# import tensorflow as tf

# train_dataset = tf.data.Dataset.from_tensor_slices((
#     dict(encoded_X_train),
#     dict(encoded_y_train)
# ))
# val_dataset = tf.data.Dataset.from_tensor_slices((
#     dict(encoded_X_test),
#     dict(encoded_y_test)
# ))
# test_dataset = tf.data.Dataset.from_tensor_slices((
#     dict(test_encodings),
#     test_labels
# ))

from torch.utils.data import Dataset

class SpiderDataset(Dataset):
  def __init__(self, sent_enc, sql_enc=None):
    self.sent_enc = sent_enc
    self.sql_enc = sql_enc

  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.sent_enc.items()}
    if self.labels:
      item["labels"] = torch.tensor(self.sql_enc[idx])
    return item

  def __len__(self):
    return len(self.sent_enc["input_ids"])



# Create torch dataset
# class CustomDataset(torch.utils.data.Dataset):
#     def __init__(self, encodings, labels=None):
#         self.encodings = encodings
#         self.labels = labels

#     def __getitem__(self, idx):
#         item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#         if self.labels:
#             item["labels"] = torch.tensor(self.labels[idx])
#         return item

#     def __len__(self):
#         return len(self.encodings["input_ids"])



In [14]:
train_dataset = SpiderDataset(encoded_X_train, encoded_y_train)
val_dataset = SpiderDataset(encoded_X_test, encoded_y_test)

In [ ]:
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [15]:
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=1,
    per_device_train_batch_size=8

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [16]:
trainer.train()

TypeError: ignored

In [17]:
print(model)

BertForPreTraining(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [ ]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

with training_args.strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

ValueError: ignored